In [7]:
import pandas as pd
from utils import all_history, transform_files
import glob
import numpy as np
import pandas as pd
import glob
# import win32com.client as win32
import jaydebeapi
from const import QUERY, CONNECT
import re



In [8]:
def read_file(pathfile,filename):
    if filename:
        if filename.endswith(".xlsx"):
            file = pd.read_excel(f'{pathfile}{filename}')
        elif filename.endswith(".csv"):
            encodings = ["utf-8", "ISO-8859-1", "latin1", "utf-16"]
            for enc in encodings:
                try:
                    file = pd.read_csv(f'{pathfile}{filename}', encoding=enc)
                    # print(f"Successfully read {filename} using {enc} encoding.")
                    return file
                except UnicodeDecodeError:
                    print(f"Encoding error with {enc}, trying another...")
            # file = pd.read_csv(f'{pathfile}{filename}', encoding='ISO-8859-1')
        return file  
    # print(assign.head())  # Display first few rows
    else:
        print("No file found in the folder.")
    

In [9]:
def clean_column_names(df):
    df.columns = df.columns.str.lower()  # Convert to lowercase
    df.columns = df.columns.str.replace(r'\.', '', regex=True)
    df.columns = df.columns.str.replace(r' ', '_', regex=True)  # Replace non-alphanumeric characters with underscores
    df.columns = df.columns.str.replace('customer_id_no|customer_no|customer_id|national_id', 'customer_no', regex=True)  # Replace variations with 'customer_no'
    df.columns = df.columns.str.replace(r'loan_no', 'contract_no', regex=True)
    df.columns = df.columns.str.replace(r'mobile.*', 'mobile_no', regex=True)
    df.columns = df.columns.str.replace(r'customer_name/surname\(thai\)', 'customer_name', regex=True)  # Replace 'customer_name/surname(thai)' with 'customer_name'
    return df    

In [10]:
conn = jaydebeapi.connect(
        CONNECT.DB,
        CONNECT.PORT,
        CONNECT.USER,
        CONNECT.CNN,
        )
cur = conn.cursor()  

query = pd.read_sql(f'''
      SELECT * FROM SIREETRON.bucket_score  
    ''', conn) 
conn.close()




C:\Users\Sireetron\AppData\Local\Temp\ipykernel_46292\629939113.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query = pd.read_sql(f'''


In [11]:
file_current = glob.glob("./input/acc_current/*") 
file_current = file_current[0].split('\\')[1]
file_current


'Acc_20250201_0830.csv'

In [12]:

print(f'file_oa_current :, {file_current} ?? ')

file_oa_current :, Acc_20250201_0830.csv ?? 


In [13]:
data_digit1 = read_file('./input/acc_current/',file_current)
data_digit1 = clean_column_names(data_digit1)
data_digit1

,basedate,contract_no,product_code,overdue_start_date,overduedays_morning,overduecnt_morning,overduedays,overduecnt,loan_amt,loan_balance,...,overduecrusage,overdueoutputtax,overduecollectionfee,overduevat,overduestamptaxamt,overduencbtaxamt,overdueetcfeeamt,lawcost,lastpaymentdt,lastpaymentamt
0,20250201,9010221122000949,A501:TL,2024-09-01,153,5:OD4,153,5:OD4,74164.44,38086.28,...,941.16,0.0,467.3,32.7,0,0,0.0,0.0,2025-01-30,3000.00
1,20250201,9010221122000931,A501:TL,,0,99:NORMAL,0,99:NORMAL,41659.76,13950.38,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2024-09-27,20971.92
2,20250201,9010221122000922,A501:TL,2023-08-31,0,98:Write-Off,0,98:Write-Off,34238.99,23851.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2023-01-31,1300.00
3,20250201,9010221122000914,A501:TL,2024-02-29,0,98:Write-Off,0,98:Write-Off,16118.33,13559.55,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2023-12-27,550.00
4,20250201,9010221122000906,A501:TL,2022-11-30,0,98:Write-Off,0,98:Write-Off,35235.40,31005.17,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2022-09-30,2100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366988,20250201,1000158090014831,A401:ML,2017-02-01,0,99:NORMAL,0,99:NORMAL,25500.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-01-31,1580.00
1366989,20250201,1000158090014814,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,24600.00,30.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-07-31,1260.00
1366990,20250201,1000158090008580,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-07-16,4000.00
1366991,20250201,1000158090005394,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-08-01,2600.00


In [14]:

# data_digit1 = pd.read_csv(f'./input/acc_current/{file_current}')
data_digit1['contract_no'] = data_digit1['contract_no'].astype(str)
data_digit1 = data_digit1[['contract_no']]

In [15]:
data_digit1

,contract_no
0,9010221122000949
1,9010221122000931
2,9010221122000922
3,9010221122000914
4,9010221122000906
...,...
1366988,1000158090014831
1366989,1000158090014814
1366990,1000158090008580
1366991,1000158090005394


In [16]:
files_history = glob.glob("./input/acc_history_monthly/*.csv") 
files_history

['./input/acc_history_monthly\\Acc_20240901_0830.csv',
 './input/acc_history_monthly\\Acc_20241001_0830.csv',
 './input/acc_history_monthly\\Acc_20241101_0830.csv',
 './input/acc_history_monthly\\Acc_20241201_0830.csv',
 './input/acc_history_monthly\\Acc_20250101_0830.csv',
 './input/acc_history_monthly\\Acc_20250201_0830.csv']

In [17]:
processed_files = []  # Store modified file names
for i in files_history:
    # print()
    file = i.split('\\')[-1] 
    # print(file)
    processed_files.append(file)  
his = transform_files(processed_files)

In [18]:
his

[['Acc_20250201_0830.csv', 'Acc_20250201_0830'],
 ['Acc_20250101_0830.csv', 'Acc_20250101_0830'],
 ['Acc_20241201_0830.csv', 'Acc_20241201_0830'],
 ['Acc_20241101_0830.csv', 'Acc_20241101_0830'],
 ['Acc_20241001_0830.csv', 'Acc_20241001_0830'],
 ['Acc_20240901_0830.csv', 'Acc_20240901_0830']]

In [19]:
for i in his[0:]:
    # print(f"Processing date: {i[0]}")
    # print((i[0], i[1]))
    data = all_history(query,i[0], i[1])
    data = clean_column_names(data)
    data_digit1 = data_digit1.merge(data, on='contract_no', how='left')

c:\Users\Sireetron\automated\kbj_automation\cscore\utils.py:19: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')
c:\Users\Sireetron\automated\kbj_automation\cscore\utils.py:19: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')
c:\Users\Sireetron\automated\kbj_automation\cscore\utils.py:19: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')


In [20]:
data_digit1

,contract_no,bucket_scoreacc_20250201_0830mnt,bucket_scoreacc_20250101_0830mnt,bucket_scoreacc_20241201_0830mnt,bucket_scoreacc_20241101_0830mnt,bucket_scoreacc_20241001_0830mnt,bucket_scoreacc_20240901_0830mnt
0,9010221122000949,5,5,5,5,5,5
1,9010221122000931,0,0,0,0,0,0
2,9010221122000922,13,13,13,13,13,13
3,9010221122000914,13,13,13,13,13,13
4,9010221122000906,13,13,13,13,13,13
...,...,...,...,...,...,...,...
1366988,1000158090014831,0,0,0,0,0,0
1366989,1000158090014814,0,0,0,0,0,0
1366990,1000158090008580,0,0,0,0,0,0
1366991,1000158090005394,0,0,0,0,0,0


In [21]:
data_performance = pd.read_csv('./input/performance_peronal_info/performance.csv')
data_performance

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_46292\295288314.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_performance = pd.read_csv('./input/performance_peronal_info/performance.csv')


,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,MOB,DIGIT2INCOME,DIGIT2AGE,DIGIT2JOB,TOTAL_DIGIT2
0,2025-03-24 00:00:00.000,7230540100522501,1650400010855,35000.0,9.0,PRODUCTION LINE LABOR,2023-05-13 00:00:00.000,22.0,40.0,3729.09,7,2,3,3,3
1,2025-03-24 00:00:00.000,7220550100000141,3230300163463,26220.0,9.0,PRODUCTION LINE LABOR,2022-05-20 00:00:00.000,34.0,48.0,94476.55,7,3,2,3,3
2,2025-03-24 00:00:00.000,7230940101266729,2810400029862,11525.0,9.0,PRODUCTION LINE LABOR,2023-09-22 00:00:00.000,18.0,39.0,5843.96,7,3,3,3,3
3,2025-03-24 00:00:00.000,9000221032000349,5300900049535,32199.0,9.0,PRODUCTION LINE LABOR,2021-03-04 00:00:00.000,48.0,45.0,7816.14,7,2,2,3,3
4,2025-03-24 00:00:00.000,9000221042012340,1310700314917,18303.0,9.0,PRODUCTION LINE LABOR,2021-04-29 00:00:00.000,46.0,25.0,2589.02,7,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418623,2025-03-24 00:00:00.000,7250370104866360,1959900141954,35000.0,6.0,SALES,2025-03-01 00:00:00.000,0.0,37.0,0.00,6,2,3,2,3
5418624,2025-03-24 00:00:00.000,7250270104797561,1509900833701,35000.0,6.0,SALES,2025-02-17 00:00:00.000,1.0,35.0,0.00,6,2,3,2,3
5418625,2025-03-24 00:00:00.000,7250370104881832,3840800081304,25000.0,10.0,GENERAL LABOR,2025-03-03 00:00:00.000,0.0,59.0,8445.77,6,3,1,2,2
5418626,2025-03-24 00:00:00.000,7250370104881833,1149900021760,25000.0,4.0,SPECIALIZATION,2025-03-03 00:00:00.000,0.0,40.0,0.00,6,3,3,1,3


In [22]:
data_performance = clean_column_names(data_performance)
# data_performance

In [23]:
data_digit1['contract_no'] = data_digit1['contract_no'].astype(str)
data_performance['contract_no']= data_performance['contract_no'].astype(str)
data_performance

,as_of_date,contract_no,customer_no,gross_income,occupation_type,occupation_name,contract_date,contract_period,age,principal_bal,mob,digit2income,digit2age,digit2job,total_digit2
0,2025-03-24 00:00:00.000,7230540100522501,1650400010855,35000.0,9.0,PRODUCTION LINE LABOR,2023-05-13 00:00:00.000,22.0,40.0,3729.09,7,2,3,3,3
1,2025-03-24 00:00:00.000,7220550100000141,3230300163463,26220.0,9.0,PRODUCTION LINE LABOR,2022-05-20 00:00:00.000,34.0,48.0,94476.55,7,3,2,3,3
2,2025-03-24 00:00:00.000,7230940101266729,2810400029862,11525.0,9.0,PRODUCTION LINE LABOR,2023-09-22 00:00:00.000,18.0,39.0,5843.96,7,3,3,3,3
3,2025-03-24 00:00:00.000,9000221032000349,5300900049535,32199.0,9.0,PRODUCTION LINE LABOR,2021-03-04 00:00:00.000,48.0,45.0,7816.14,7,2,2,3,3
4,2025-03-24 00:00:00.000,9000221042012340,1310700314917,18303.0,9.0,PRODUCTION LINE LABOR,2021-04-29 00:00:00.000,46.0,25.0,2589.02,7,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418623,2025-03-24 00:00:00.000,7250370104866360,1959900141954,35000.0,6.0,SALES,2025-03-01 00:00:00.000,0.0,37.0,0.00,6,2,3,2,3
5418624,2025-03-24 00:00:00.000,7250270104797561,1509900833701,35000.0,6.0,SALES,2025-02-17 00:00:00.000,1.0,35.0,0.00,6,2,3,2,3
5418625,2025-03-24 00:00:00.000,7250370104881832,3840800081304,25000.0,10.0,GENERAL LABOR,2025-03-03 00:00:00.000,0.0,59.0,8445.77,6,3,1,2,2
5418626,2025-03-24 00:00:00.000,7250370104881833,1149900021760,25000.0,4.0,SPECIALIZATION,2025-03-03 00:00:00.000,0.0,40.0,0.00,6,3,3,1,3


In [24]:
data_digit1

,contract_no,bucket_scoreacc_20250201_0830mnt,bucket_scoreacc_20250101_0830mnt,bucket_scoreacc_20241201_0830mnt,bucket_scoreacc_20241101_0830mnt,bucket_scoreacc_20241001_0830mnt,bucket_scoreacc_20240901_0830mnt
0,9010221122000949,5,5,5,5,5,5
1,9010221122000931,0,0,0,0,0,0
2,9010221122000922,13,13,13,13,13,13
3,9010221122000914,13,13,13,13,13,13
4,9010221122000906,13,13,13,13,13,13
...,...,...,...,...,...,...,...
1366988,1000158090014831,0,0,0,0,0,0
1366989,1000158090014814,0,0,0,0,0,0
1366990,1000158090008580,0,0,0,0,0,0
1366991,1000158090005394,0,0,0,0,0,0


In [25]:
data_digit1['avg_digit1'] = data_digit1.loc[:, data_digit1.columns.str.startswith('bucket_score')].apply(pd.to_numeric, errors='coerce').sum(axis=1).fillna(0).astype(int)/6


In [26]:
data_digit1

,contract_no,bucket_scoreacc_20250201_0830mnt,bucket_scoreacc_20250101_0830mnt,bucket_scoreacc_20241201_0830mnt,bucket_scoreacc_20241101_0830mnt,bucket_scoreacc_20241001_0830mnt,bucket_scoreacc_20240901_0830mnt,avg_digit1
0,9010221122000949,5,5,5,5,5,5,5.0
1,9010221122000931,0,0,0,0,0,0,0.0
2,9010221122000922,13,13,13,13,13,13,13.0
3,9010221122000914,13,13,13,13,13,13,13.0
4,9010221122000906,13,13,13,13,13,13,13.0
...,...,...,...,...,...,...,...,...
1366988,1000158090014831,0,0,0,0,0,0,0.0
1366989,1000158090014814,0,0,0,0,0,0,0.0
1366990,1000158090008580,0,0,0,0,0,0,0.0
1366991,1000158090005394,0,0,0,0,0,0,0.0


In [27]:
data_digit1['avg_digit1'] = data_digit1.loc[:, data_digit1.columns.str.startswith('BUCKET_SCORE')].apply(pd.to_numeric, errors='coerce').sum(axis=1).fillna(0).astype(int)/6


In [28]:
data_all_digit = data_performance.merge(data_digit1, left_on='contract_no', right_on='contract_no', how='left')
# data_all_digit['AVG_DIGIT1'] = data_all_digit.loc[:, data_all_digit.columns.str.startswith('SCORE_INDEX')].apply(pd.to_numeric, errors='coerce').mean(axis=1).fillna(0).astype(int)
data_all_digit['mob'] = pd.to_numeric(data_all_digit['mob'], errors='coerce').fillna(0).astype(int)
data_all_digit

,as_of_date,contract_no,customer_no,gross_income,occupation_type,occupation_name,contract_date,contract_period,age,principal_bal,...,digit2age,digit2job,total_digit2,bucket_scoreacc_20250201_0830mnt,bucket_scoreacc_20250101_0830mnt,bucket_scoreacc_20241201_0830mnt,bucket_scoreacc_20241101_0830mnt,bucket_scoreacc_20241001_0830mnt,bucket_scoreacc_20240901_0830mnt,avg_digit1
0,2025-03-24 00:00:00.000,7230540100522501,1650400010855,35000.0,9.0,PRODUCTION LINE LABOR,2023-05-13 00:00:00.000,22.0,40.0,3729.09,...,3,3,3,13,13,9,8,7,6,0.0
1,2025-03-24 00:00:00.000,7220550100000141,3230300163463,26220.0,9.0,PRODUCTION LINE LABOR,2022-05-20 00:00:00.000,34.0,48.0,94476.55,...,2,3,3,13,13,13,13,13,13,0.0
2,2025-03-24 00:00:00.000,7230940101266729,2810400029862,11525.0,9.0,PRODUCTION LINE LABOR,2023-09-22 00:00:00.000,18.0,39.0,5843.96,...,3,3,3,0,0,0,0,0,0,0.0
3,2025-03-24 00:00:00.000,9000221032000349,5300900049535,32199.0,9.0,PRODUCTION LINE LABOR,2021-03-04 00:00:00.000,48.0,45.0,7816.14,...,2,3,3,13,13,13,13,13,13,0.0
4,2025-03-24 00:00:00.000,9000221042012340,1310700314917,18303.0,9.0,PRODUCTION LINE LABOR,2021-04-29 00:00:00.000,46.0,25.0,2589.02,...,3,3,3,13,13,13,13,13,13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418623,2025-03-24 00:00:00.000,7250370104866360,1959900141954,35000.0,6.0,SALES,2025-03-01 00:00:00.000,0.0,37.0,0.00,...,3,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5418624,2025-03-24 00:00:00.000,7250270104797561,1509900833701,35000.0,6.0,SALES,2025-02-17 00:00:00.000,1.0,35.0,0.00,...,3,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5418625,2025-03-24 00:00:00.000,7250370104881832,3840800081304,25000.0,10.0,GENERAL LABOR,2025-03-03 00:00:00.000,0.0,59.0,8445.77,...,1,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5418626,2025-03-24 00:00:00.000,7250370104881833,1149900021760,25000.0,4.0,SPECIALIZATION,2025-03-03 00:00:00.000,0.0,40.0,0.00,...,3,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
files_tdr = glob.glob("./Input/tdr/*.*")
files_tdr

['./Input/tdr\\TDR.xlsx']

In [34]:
df_list = []  # Store individual DataFrames
for file in files_tdr:
    # print(f"Reading {file}...")
    path = glob.glob(f"{file}")[0].split('\\')[-1]
    # print(path)
    df = read_file('./input/tdr/',path)
    df_list.append(df)

In [35]:
data_tdr = pd.concat(df_list, ignore_index=True)
data_tdr = clean_column_names(data_tdr)
data_tdr['contract_no'] = data_tdr['contract_no'].astype(str).apply(lambda x: re.sub(r'\D', '', x))
data_tdr = data_tdr.loc[
    (data_tdr['product'] == 'TL') & 
    (
        data_tdr['project_code_name'].str.contains('ปรับโครงสร้าง', na=False) | 
        data_tdr['project_code_name'].str.contains('TDR', na=False) | 
        data_tdr['project_code_name'].str.contains('DR', na=False)
    )
].reset_index(drop=True)[['contract_no','project_code_name']]

In [36]:
data_cscore = data_all_digit.merge(data_tdr, left_on='contract_no', right_on='contract_no', how='left')
data_cscore

,as_of_date,contract_no,customer_no,gross_income,occupation_type,occupation_name,contract_date,contract_period,age,principal_bal,...,digit2job,total_digit2,bucket_scoreacc_20250201_0830mnt,bucket_scoreacc_20250101_0830mnt,bucket_scoreacc_20241201_0830mnt,bucket_scoreacc_20241101_0830mnt,bucket_scoreacc_20241001_0830mnt,bucket_scoreacc_20240901_0830mnt,avg_digit1,project_code_name
0,2025-03-24 00:00:00.000,7230540100522501,1650400010855,35000.0,9.0,PRODUCTION LINE LABOR,2023-05-13 00:00:00.000,22.0,40.0,3729.09,...,3,3,13,13,9,8,7,6,0.0,NaN
1,2025-03-24 00:00:00.000,7220550100000141,3230300163463,26220.0,9.0,PRODUCTION LINE LABOR,2022-05-20 00:00:00.000,34.0,48.0,94476.55,...,3,3,13,13,13,13,13,13,0.0,NaN
2,2025-03-24 00:00:00.000,7230940101266729,2810400029862,11525.0,9.0,PRODUCTION LINE LABOR,2023-09-22 00:00:00.000,18.0,39.0,5843.96,...,3,3,0,0,0,0,0,0,0.0,NaN
3,2025-03-24 00:00:00.000,9000221032000349,5300900049535,32199.0,9.0,PRODUCTION LINE LABOR,2021-03-04 00:00:00.000,48.0,45.0,7816.14,...,3,3,13,13,13,13,13,13,0.0,NaN
4,2025-03-24 00:00:00.000,9000221042012340,1310700314917,18303.0,9.0,PRODUCTION LINE LABOR,2021-04-29 00:00:00.000,46.0,25.0,2589.02,...,3,3,13,13,13,13,13,13,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418623,2025-03-24 00:00:00.000,7250370104866360,1959900141954,35000.0,6.0,SALES,2025-03-01 00:00:00.000,0.0,37.0,0.00,...,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5418624,2025-03-24 00:00:00.000,7250270104797561,1509900833701,35000.0,6.0,SALES,2025-02-17 00:00:00.000,1.0,35.0,0.00,...,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5418625,2025-03-24 00:00:00.000,7250370104881832,3840800081304,25000.0,10.0,GENERAL LABOR,2025-03-03 00:00:00.000,0.0,59.0,8445.77,...,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5418626,2025-03-24 00:00:00.000,7250370104881833,1149900021760,25000.0,4.0,SPECIALIZATION,2025-03-03 00:00:00.000,0.0,40.0,0.00,...,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Define conditions
conditions = [
    data_cscore['project_code_name'].notna(),      # A is not null
    data_cscore['mob'] < 7,          # A is less than 7
    data_cscore['avg_digit1'] > 3  ,
    data_cscore['avg_digit1'] > 1,# C is equal to 3
    data_cscore['avg_digit1'] >= 0,
]

In [38]:
# Define corresponding value
choices = ['H', 'D', 'H','M','L']
data_cscore['total_digit1']= np.select(conditions, choices, default='Other')
data_cscore['total_digit2'] = data_cscore['total_digit2'].map({3: 'H', 2: 'M', 1: 'L'})
data_cscore['final_score'] =data_cscore['total_digit1'].astype(str)+ data_cscore['total_digit2'].astype(str) +'0'+ data_cscore['mob'].astype(str)
data_cscore

,as_of_date,contract_no,customer_no,gross_income,occupation_type,occupation_name,contract_date,contract_period,age,principal_bal,...,bucket_scoreacc_20250201_0830mnt,bucket_scoreacc_20250101_0830mnt,bucket_scoreacc_20241201_0830mnt,bucket_scoreacc_20241101_0830mnt,bucket_scoreacc_20241001_0830mnt,bucket_scoreacc_20240901_0830mnt,avg_digit1,project_code_name,total_digit1,final_score
0,2025-03-24 00:00:00.000,7230540100522501,1650400010855,35000.0,9.0,PRODUCTION LINE LABOR,2023-05-13 00:00:00.000,22.0,40.0,3729.09,...,13,13,9,8,7,6,0.0,NaN,L,LH07
1,2025-03-24 00:00:00.000,7220550100000141,3230300163463,26220.0,9.0,PRODUCTION LINE LABOR,2022-05-20 00:00:00.000,34.0,48.0,94476.55,...,13,13,13,13,13,13,0.0,NaN,L,LH07
2,2025-03-24 00:00:00.000,7230940101266729,2810400029862,11525.0,9.0,PRODUCTION LINE LABOR,2023-09-22 00:00:00.000,18.0,39.0,5843.96,...,0,0,0,0,0,0,0.0,NaN,L,LH07
3,2025-03-24 00:00:00.000,9000221032000349,5300900049535,32199.0,9.0,PRODUCTION LINE LABOR,2021-03-04 00:00:00.000,48.0,45.0,7816.14,...,13,13,13,13,13,13,0.0,NaN,L,LH07
4,2025-03-24 00:00:00.000,9000221042012340,1310700314917,18303.0,9.0,PRODUCTION LINE LABOR,2021-04-29 00:00:00.000,46.0,25.0,2589.02,...,13,13,13,13,13,13,0.0,NaN,L,LH07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418623,2025-03-24 00:00:00.000,7250370104866360,1959900141954,35000.0,6.0,SALES,2025-03-01 00:00:00.000,0.0,37.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DH06
5418624,2025-03-24 00:00:00.000,7250270104797561,1509900833701,35000.0,6.0,SALES,2025-02-17 00:00:00.000,1.0,35.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DH06
5418625,2025-03-24 00:00:00.000,7250370104881832,3840800081304,25000.0,10.0,GENERAL LABOR,2025-03-03 00:00:00.000,0.0,59.0,8445.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DM06
5418626,2025-03-24 00:00:00.000,7250370104881833,1149900021760,25000.0,4.0,SPECIALIZATION,2025-03-03 00:00:00.000,0.0,40.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DH06


In [49]:
data_cscore.columns

Index(['as_of_date', 'contract_no', 'customer_no', 'gross_income',
       'occupation_type', 'occupation_name', 'contract_date',
       'contract_period', 'age', 'principal_bal', 'mob', 'digit2income',
       'digit2age', 'digit2job', 'total_digit2',
       'bucket_scoreacc_20250201_0830mnt', 'bucket_scoreacc_20250101_0830mnt',
       'bucket_scoreacc_20241201_0830mnt', 'bucket_scoreacc_20241101_0830mnt',
       'bucket_scoreacc_20241001_0830mnt', 'bucket_scoreacc_20240901_0830mnt',
       'avg_digit1', 'project_code_name', 'total_digit1', 'final_score'],
      dtype='object')

In [52]:
data_cscore[:100].to_csv('output.csv', index=False)

In [53]:
ddd = pd.read_csv('./output.csv')

In [54]:
ddd

,as_of_date,contract_no,customer_no,gross_income,occupation_type,occupation_name,contract_date,contract_period,age,principal_bal,...,bucket_scoreacc_20250201_0830mnt,bucket_scoreacc_20250101_0830mnt,bucket_scoreacc_20241201_0830mnt,bucket_scoreacc_20241101_0830mnt,bucket_scoreacc_20241001_0830mnt,bucket_scoreacc_20240901_0830mnt,avg_digit1,project_code_name,total_digit1,final_score
0,2025-03-24 00:00:00.000,7230540100522501,1650400010855,35000.0,9.0,PRODUCTION LINE LABOR,2023-05-13 00:00:00.000,22.0,40.0,3729.09,...,13.0,13.0,9.0,8.0,7.0,6.0,0.0,NaN,L,LH07
1,2025-03-24 00:00:00.000,7220550100000141,3230300163463,26220.0,9.0,PRODUCTION LINE LABOR,2022-05-20 00:00:00.000,34.0,48.0,94476.55,...,13.0,13.0,13.0,13.0,13.0,13.0,0.0,NaN,L,LH07
2,2025-03-24 00:00:00.000,7230940101266729,2810400029862,11525.0,9.0,PRODUCTION LINE LABOR,2023-09-22 00:00:00.000,18.0,39.0,5843.96,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,L,LH07
3,2025-03-24 00:00:00.000,9000221032000349,5300900049535,32199.0,9.0,PRODUCTION LINE LABOR,2021-03-04 00:00:00.000,48.0,45.0,7816.14,...,13.0,13.0,13.0,13.0,13.0,13.0,0.0,NaN,L,LH07
4,2025-03-24 00:00:00.000,9000221042012340,1310700314917,18303.0,9.0,PRODUCTION LINE LABOR,2021-04-29 00:00:00.000,46.0,25.0,2589.02,...,13.0,13.0,13.0,13.0,13.0,13.0,0.0,NaN,L,LH07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-03-24 00:00:00.000,7230940101275280,1330400452397,16492.0,5.0,OFFICE WORK,2023-09-24 00:00:00.000,18.0,29.0,5389.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,L,LH07
96,2025-03-24 00:00:00.000,7220350100000163,1120100130281,9570.0,10.0,GENERAL LABOR,2022-03-22 00:00:00.000,36.0,37.0,38774.37,...,13.0,13.0,13.0,13.0,13.0,13.0,0.0,NaN,L,LH07
97,2025-03-24 00:00:00.000,7220250100000431,1103701062151,15430.0,10.0,GENERAL LABOR,2022-02-28 00:00:00.000,36.0,31.0,37542.62,...,13.0,13.0,13.0,13.0,13.0,13.0,0.0,NaN,L,LH07
98,2025-03-24 00:00:00.000,7220940100000729,1329400012735,15738.0,9.0,PRODUCTION LINE LABOR,2022-09-12 00:00:00.000,30.0,22.0,5851.24,...,13.0,13.0,13.0,13.0,13.0,13.0,0.0,NaN,L,LH07


In [69]:
file_assign = glob.glob("./input/assign_data/*.xlsx") 

In [73]:
len(file_assign)<1

False

In [48]:
data_assign = pd.read_excel(file_assign[0])
data_assign = clean_column_names(data_assign)
data_assign = data_assign[['contract_no','old_oa','new_oa','overdue_months_division_code']]
data_assign['contract_no'] = data_assign['contract_no'].astype(str)
data_assign

,contract_no,old_oa,new_oa,overdue_months_division_code
0,7220730100003250,KOR,KOR,Grace_X-Day
1,7221030100003286,NaN,Delay calling,Grace_X-Day
2,7230340100241933,Delay calling,Delay calling,Grace_X-Day
3,7230530100570947,PCN,KOR,OD2
4,7230940101310936,NaN,Delay calling,Grace_X-Day
...,...,...,...,...
40416,7240450102627362,KWC,KWC,OD6
40417,7240550102879930,GMV,MMN,OD6
40418,7240750103341461,KWC,KWC,OD6
40419,7240650103061703,MMN,KWC,OD6


---

In [34]:
data_digit1['contract_no'] = data_digit1['contract_no'].astype(str)
data_performance['contract_no']= data_performance['contract_no'].astype(str)
data_performance

,as_of_date,contract_no,customer_no,gross_income,occupation_type,occupation_name,contract_date,contract_period,age,principal_bal,mob,digit2income,digit2age,digit2job,total_digit2
0,2025-03-19 00:00:00.000,5000161051018231,1110300202706,9750.0,9.0,PRODUCTION LINE LABOR,2018-05-07 00:00:00.000,82.0,27.0,0.00,7,3,3,3,3
1,2025-03-19 00:00:00.000,1000161051042121,1110300202706,9750.0,9.0,PRODUCTION LINE LABOR,2018-05-14 00:00:00.000,82.0,27.0,801.00,7,3,3,3,3
2,2025-03-19 00:00:00.000,5002461061050530,1110300203281,10140.0,5.0,OFFICE WORK,2018-06-13 00:00:00.000,81.0,27.0,0.00,7,3,3,1,3
3,2025-03-19 00:00:00.000,5000361021006968,1110300205578,9300.0,10.0,GENERAL LABOR,2018-02-02 00:00:00.000,85.0,27.0,13890.67,7,3,3,2,3
4,2025-03-19 00:00:00.000,5000461041075891,1110300208135,13737.0,9.0,PRODUCTION LINE LABOR,2018-04-26 00:00:00.000,82.0,27.0,0.00,7,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5386968,2025-03-19 00:00:00.000,7250170104680761,3800400714270,12500.0,4.0,SPECIALIZATION,2025-01-31 00:00:00.000,1.0,41.0,0.00,6,3,2,1,2
5386969,2025-03-19 00:00:00.000,7241270104483495,1301400032895,17500.0,5.0,OFFICE WORK,2024-12-28 00:00:00.000,2.0,39.0,0.00,6,3,3,1,3
5386970,2025-03-19 00:00:00.000,7250170104615532,5361200029818,50500.0,2.0,UNREGISTERED BUSINESS OWNER,2025-01-19 00:00:00.000,2.0,59.0,24211.25,6,2,1,3,2
5386971,2025-03-19 00:00:00.000,7250170104600813,3720200607489,17500.0,9.0,PRODUCTION LINE LABOR,2025-01-16 00:00:00.000,2.0,46.0,0.00,6,3,2,3,3


In [27]:
data_performance.columns

Index(['as_of_date', 'contract_no', 'customer_no', 'gross_income',
       'occupation_type', 'occupation_name', 'contract_no', 'contract_no',
       'age', 'principal_bal', 'mob', 'digit2income', 'digit2age', 'digit2job',
       'total_digit2'],
      dtype='object')

In [25]:
data_digit1['avg_digit1'] = data_digit1.loc[:, data_digit1.columns.str.startswith('BUCKET_SCORE')].apply(pd.to_numeric, errors='coerce').sum(axis=1).fillna(0).astype(int)/6
data_all_digit = data_performance.merge(data_digit1, left_on='contract_no', right_on='contract_no', how='left', suffixes=('_df1', '_df2'))
data_all_digit


ValueError: The column label 'contract_no' is not unique.

In [ ]:
# data_all_digit['AVG_DIGIT1'] = data_all_digit.loc[:, data_all_digit.columns.str.startswith('SCORE_INDEX')].apply(pd.to_numeric, errors='coerce').mean(axis=1).fillna(0).astype(int)

data_all_digit['mob'] = pd.to_numeric(data_all_digit['mob'], errors='coerce').fillna(0).astype(int)
data_all_digit

In [5]:
for i in his[0:]:
    # print(f"Processing date: {i[0]}")
    print((i[0], i[1]))
    data = all_history(query,i[0], i[1])
    data_digit1 = data_digit1.merge(data, on='Loan No', how='left')

('Acc_20250301_0830.csv', 'Acc_20250301_0830')
('Acc_20250201_0830.csv', 'Acc_20250201_0830')
('Acc_20250101_0830.csv', 'Acc_20250101_0830')
('Acc_20241201_0830.csv', 'Acc_20241201_0830')
('Acc_20241101_0830.csv', 'Acc_20241101_0830')


c:\Users\Sireetron\automated\kbj_automation\Performance\utils.py:19: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')


('Acc_20241001_0830.csv', 'Acc_20241001_0830')


c:\Users\Sireetron\automated\kbj_automation\Performance\utils.py:19: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')


In [6]:
data_digit1

,Loan No,SCORE_INDEXAcc_20250301_0830mnt,SCORE_INDEXAcc_20250201_0830mnt,SCORE_INDEXAcc_20250101_0830mnt,SCORE_INDEXAcc_20241201_0830mnt,SCORE_INDEXAcc_20241101_0830mnt,SCORE_INDEXAcc_20241001_0830mnt
0,9010221122000949,3,3,3,3,3,3
1,9010221122000931,1,1,1,1,1,1
2,9010221122000922,3,3,3,3,3,3
3,9010221122000914,3,3,3,3,3,3
4,9010221122000906,3,3,3,3,3,3
...,...,...,...,...,...,...,...
1372970,1000158090014831,1,1,1,1,1,1
1372971,1000158090014814,1,1,1,1,1,1
1372972,1000158090008580,1,1,1,1,1,1
1372973,1000158090005394,1,1,1,1,1,1


In [15]:
# performane digit2
data_performance = pd.read_csv('./input/performance/performance.csv')
data_digit1['Loan No'] = data_digit1['Loan No'].astype(str)
data_performance['CONTRACT_NO']= data_performance['CONTRACT_NO'].astype(str)
data_digit1['AVG_DIGIT1'] = data_digit1.loc[:, data_digit1.columns.str.startswith('BUCKET_SCORE')].apply(pd.to_numeric, errors='coerce').sum(axis=1).fillna(0).astype(int)/6
data_all_digit = data_performance.merge(data_digit1, left_on='CONTRACT_NO', right_on='Loan No', how='left')
# data_all_digit['AVG_DIGIT1'] = data_all_digit.loc[:, data_all_digit.columns.str.startswith('SCORE_INDEX')].apply(pd.to_numeric, errors='coerce').mean(axis=1).fillna(0).astype(int)
data_all_digit['MOB'] = pd.to_numeric(data_all_digit['MOB'], errors='coerce').fillna(0).astype(int)



C:\Users\Sireetron\AppData\Local\Temp\ipykernel_16276\3607971684.py:2: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_performance = pd.read_csv('./input/performance/performance.csv')


In [16]:
data_performance

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,MOB,DIGIT2INCOME,DIGIT2AGE,DIGIT2JOB,TOTAL_DIGIT2
0,2025-03-09 00:00:00.000,5000160071029814,3450800190247,10500.0,10.0,GENERAL LABOR,2017-07-17 00:00:00.000,91.0,48.0,0.00,7,3,2,2,3
1,2025-03-09 00:00:00.000,7221140100022427,3601000365947,25763.0,9.0,PRODUCTION LINE LABOR,2022-11-11 00:00:00.000,28.0,43.0,31783.61,7,3,2,3,3
2,2025-03-09 00:00:00.000,7230950101266462,1709700073957,30000.0,5.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,47052.21,7,3,3,1,3
3,2025-03-09 00:00:00.000,5000161041003419,3300800674709,15611.0,9.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,0.00,7,3,2,3,3
4,2025-03-09 00:00:00.000,7230410100429114,1401500153197,12224.0,9.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,274168.44,7,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104792037,1720200122632,45000.0,4.0,SPECIALIZATION,2025-02-16 00:00:00.000,0.0,29.0,0.00,6,2,3,1,2
5323432,2025-03-09 00:00:00.000,7250370104881671,3650100771393,17500.0,9.0,PRODUCTION LINE LABOR,2025-03-02 00:00:00.000,0.0,63.0,0.00,6,3,1,3,3
5323433,2025-03-09 00:00:00.000,7250370104881678,1200400034464,35000.0,8.0,SERVICE JOB,2025-03-02 00:00:00.000,0.0,36.0,0.00,6,2,3,2,3
5323434,2025-03-09 00:00:00.000,7250270104729446,1679900263742,45000.0,5.0,OFFICE WORK,2025-02-06 00:00:00.000,1.0,32.0,44944.74,6,2,3,1,2


In [17]:
data_all_digit.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'NATIONAL_ID', 'GROSS_INCOME',
       'OCCUPATION_TYPE', 'OCCUPATION_NAME', 'CONTRACT_DATE',
       'CONTRACT_PERIOD', 'AGE', 'PRINCIPAL_BAL', 'MOB', 'DIGIT2INCOME',
       'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2', 'Loan No',
       'SCORE_INDEXAcc_20250301_0830mnt', 'SCORE_INDEXAcc_20250201_0830mnt',
       'SCORE_INDEXAcc_20250101_0830mnt', 'SCORE_INDEXAcc_20241201_0830mnt',
       'SCORE_INDEXAcc_20241101_0830mnt', 'SCORE_INDEXAcc_20241001_0830mnt',
       'AVG_DIGIT1'],
      dtype='object')

In [18]:
# TDH digit2
files_tdr = glob.glob("./Input/ar_all/*.*")
print(f'file_files_tdr:, {files_tdr} ?? ')

df_list = []  # Store individual DataFrames
for file in files_tdr:
    # print(f"Reading {file}...")
    df = pd.read_csv(file, encoding='cp874', low_memory=False)
    df_list.append(df)
data_tdr = pd.concat(df_list, ignore_index=True)

data_tdr['Loan No'] = data_tdr['Loan No'].astype(str).apply(lambda x: re.sub(r'\D', '', x))
data_tdr = data_tdr.loc[
    (data_tdr['Product'] == 'TL') & 
    (
        data_tdr['Project Code Name'].str.contains('ปรับโครงสร้าง', na=False) | 
        data_tdr['Project Code Name'].str.contains('TDR', na=False) | 
        data_tdr['Project Code Name'].str.contains('DR', na=False)
    )
].reset_index(drop=True)[['Loan No','Project Code Name']]


file_files_tdr:, ['./Input/ar_all\\AR_ALL_20250310_1.csv', './Input/ar_all\\AR_ALL_20250310_2.csv'] ?? 


In [19]:
data_cscore = data_all_digit.merge(data_tdr, left_on='CONTRACT_NO', right_on='Loan No', how='left')


In [20]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,...,Loan No_x,SCORE_INDEXAcc_20250301_0830mnt,SCORE_INDEXAcc_20250201_0830mnt,SCORE_INDEXAcc_20250101_0830mnt,SCORE_INDEXAcc_20241201_0830mnt,SCORE_INDEXAcc_20241101_0830mnt,SCORE_INDEXAcc_20241001_0830mnt,AVG_DIGIT1,Loan No_y,Project Code Name
0,2025-03-09 00:00:00.000,5000160071029814,3450800190247,10500.0,10.0,GENERAL LABOR,2017-07-17 00:00:00.000,91.0,48.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-03-09 00:00:00.000,7221140100022427,3601000365947,25763.0,9.0,PRODUCTION LINE LABOR,2022-11-11 00:00:00.000,28.0,43.0,31783.61,...,7221140100022427,3,3,3,3,3,3,0.0,NaN,NaN
2,2025-03-09 00:00:00.000,7230950101266462,1709700073957,30000.0,5.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,47052.21,...,7230950101266462,2,1,1,3,3,3,0.0,7230950101266462,ปรับโครงสร้างหนี้_DR_Collection_15%
3,2025-03-09 00:00:00.000,5000161041003419,3300800674709,15611.0,9.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-03-09 00:00:00.000,7230410100429114,1401500153197,12224.0,9.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,274168.44,...,7230410100429114,3,3,3,3,3,3,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104792037,1720200122632,45000.0,4.0,SPECIALIZATION,2025-02-16 00:00:00.000,0.0,29.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5323432,2025-03-09 00:00:00.000,7250370104881671,3650100771393,17500.0,9.0,PRODUCTION LINE LABOR,2025-03-02 00:00:00.000,0.0,63.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5323433,2025-03-09 00:00:00.000,7250370104881678,1200400034464,35000.0,8.0,SERVICE JOB,2025-03-02 00:00:00.000,0.0,36.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5323434,2025-03-09 00:00:00.000,7250270104729446,1679900263742,45000.0,5.0,OFFICE WORK,2025-02-06 00:00:00.000,1.0,32.0,44944.74,...,7250270104729446,1,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [21]:
data_cscore['AVG_DIGIT1']

0          NaN
1          0.0
2          0.0
3          NaN
4          0.0
          ... 
5323431    NaN
5323432    NaN
5323433    NaN
5323434    0.0
5323435    0.0
Name: AVG_DIGIT1, Length: 5323436, dtype: float64